# Training Environment

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch  # Package name: torch (for pip), pytorch (for conda)
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

class BasicDataset(data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __getitem__(self, idx):
        return dict(X=self.X[idx], y=self.y[idx])
    
    def __len__(self):
        return self.X.shape[0]

## Prepare Data

In [2]:
import torchvision.datasets as datasets

mnist = False
cifar10 = True
cifar100 = False
assert mnist ^ cifar10 ^ cifar100

n_classes = 10
if mnist:
    trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
    testset = datasets.MNIST(root='./data', train=False, download=True, transform=None)
if cifar10:
    trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=None)
    testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=None)
if cifar100:
    n_classes = 100
    trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=None)
    testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=None)

print(trainset)
print(testset)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test


In [3]:
def to_rgb(x_grey: torch.Tensor) -> torch.Tensor:
    if len(x_grey.size()) == 3:
        helper = torch.unsqueeze(x_grey, 1)
        return helper.repeat(1, 3, 1, 1).float()
    elif len(x_grey.size()) == 4 and x_grey.size()[1] == 1:
        return x_grey.repeat(1, 3, 1, 1).float()
    elif len(x_grey.size()) == 4 and x_grey.size()[1] == 3:
        return x_grey
    elif len(x_grey.size()) == 4:
        raise ValueError(f'The size of this image tensor is not valid.\
        A 4th order image tensor must have dim1==1 (grey-scale) or dim1==3 (rgb).\
        Unknown format cannot be transformed to rgb.')
    else:
        raise ValueError(f'The size of this image-tensor is not valid.\
        Must be either 3rd (grey-scale) order tensor or 4th order tensor (rgb).\
        Got order {len(x_grey.size())}')
        
def swap_data(X):
    X1 = np.swapaxes(X, 1, 3)
    X2 = np.swapaxes(X1, 2, 3)
    return X2

if mnist:
    X_train_grey = trainset.train_data
    X_train = to_rgb(X_train_grey)
    X_test_grey = testset.test_data
    X_test = to_rgb(X_test_grey)
    y_train = trainset.train_labels
    y_test = testset.test_labels
else:
    X_train = torch.tensor(swap_data(trainset.data))
    y_train = torch.tensor(trainset.targets)
    X_test = torch.tensor(swap_data(testset.data))
    y_test = torch.tensor(testset.targets)
    
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

X_train: torch.Size([50000, 3, 32, 32])
y_train: torch.Size([50000])
X_test: torch.Size([10000, 3, 32, 32])
y_test: torch.Size([10000])


## Model load, modifications and GPU optimization

In [4]:
from torchvision import models

'''
model = models.resnet18(pretrained=False) # set model here
in_ftr = model.fc.in_features
out_ftr = n_classes
model.fc = nn.Linear(in_ftr,out_ftr,bias=True)
'''

# redefining stuff for AdaSecant test runs
X_train = torch.rand(256, 10)
X_test = torch.rand(256, 10)
y_train = torch.randint(10, (256,))
y_test = torch.randint(10, (256,))

print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

model = torch.nn.Sequential(nn.Linear(10, 10), nn.Linear(10, 10))
for param in model.parameters():
    print(param)

dataset_train = BasicDataset(X_train, y_train)
dataset_test = BasicDataset(X_test, y_test)

X_train: torch.Size([256, 10])
y_train: torch.Size([256])
X_test: torch.Size([256, 10])
y_test: torch.Size([256])
Parameter containing:
tensor([[ 0.1019, -0.2096,  0.2387,  0.0221, -0.1699, -0.2565,  0.0515,  0.0049,
          0.0977,  0.0101],
        [ 0.0609, -0.2976,  0.3077,  0.0428,  0.2950, -0.2029, -0.1542,  0.2809,
          0.2241,  0.0883],
        [ 0.1602, -0.2710,  0.0379, -0.2156,  0.0612, -0.0176, -0.1572, -0.2604,
          0.2353,  0.0274],
        [-0.0295,  0.2863,  0.0900,  0.1534, -0.0436,  0.0571, -0.2137, -0.2722,
         -0.2351,  0.0856],
        [ 0.0831, -0.0030,  0.1813, -0.1638, -0.1063,  0.0943,  0.2320, -0.1975,
          0.2014,  0.2423],
        [ 0.1700,  0.2467,  0.0558, -0.1215,  0.2615, -0.1175, -0.1780,  0.2268,
          0.0432, -0.2874],
        [ 0.2810,  0.0059,  0.1897, -0.2780, -0.1360,  0.2281,  0.0259,  0.2023,
          0.0980,  0.1981],
        [-0.1903,  0.1915, -0.1181,  0.2009, -0.0194,  0.1523,  0.1199, -0.2610,
         -0.2811,  0

Check specs for GPU-based training.

In [5]:
print('cuda available:', torch.cuda.is_available())
print('cuda device name:', torch.cuda.get_device_name())
print('cuda device id', torch.cuda.current_device())

cuda available: True
cuda device name: NVIDIA GeForce 940MX
cuda device id 0


Move model and data to GPU.

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
X_train.to(device)
print('worked?', X_train.to(device).is_cuda)

worked? True


## Implementing AdaSecant
https://github.com/pytorch/pytorch/blob/master/torch/optim/sgd.py

In [7]:
from typing import List

class AdaSecant(optim.Optimizer):
    r"""Documentation
    Basis copied from https://github.com/pytorch/pytorch/blob/master/torch/optim/sgd.py.
    Left out closure, momentum-related stuff, __setstate__ as it does not seem to be necessary.
    """

    def __init__(self, params, lr=None):
        if lr is not None:
            print('Warning: lr is not a parameter for AdaSecant. Your lr will be set to None')
            lr = None
        defaults = dict(lr=lr)
        super().__init__(params, defaults)
        self.ready = False
        self.current_gradients = None
        # here we need to introduce some attribute to save params/gradients from old batch.
        self.moving_average_of_g = None
        self.delta = None
        self.memory_size = None
        self.gradients = None
        
    @torch.no_grad()
    def pre_step(self):
        for group in self.param_groups:
            d_p_list = []
        
            for p in group['params']:
                    # subgrouping of parameters for each layer, bias and weights separately (each tensor)
                    if p.grad is not None:
                        d_p_list.append(p.grad)
        
        self.current_gradients = d_p_list
        self.ready = True
        pass

    @torch.no_grad()
    def step(self):
        """Performs a single optimization step.
        """
        
        if not self.ready:
            raise RuntimeError('You must perform optimizer.pre_step() before performing ' +
                               'optimizer.step() when using AdaSecant.\n' +
                               'pre_step ensures that the gradient is saved while step will generate the ' +
                               'gradient on the new batch.\n' +
                               'Recommended call sequence:' +
                               ' \n model.zero_grad(), \n loss = ..., \n loss.backwards(), \n ' +
                               'optimizer.pre_step(), \n model.zero_grad(), \n loss = ..., \n ' +
                               'loss.backwards(), \n optimizer.step()')
        else:
            self.ready = False

        for group in self.param_groups:
            # all parameters of a model seem to be contained within the same group
            params_with_grad = []
            next_gradients = []

            for p in group['params']:
                # subgrouping of parameters for each layer, bias and weights separately (each tensor)
                if p.grad is not None:
                    params_with_grad.append(p)
                    next_gradients.append(p.grad)
            
            print('current')
            print(self.current_gradients)
            print('next')
            print(next_gradients)
            adasecant(self, params_with_grad, next_gradients)
            
        return #loss
    
def adasecant(optimizer: AdaSecant, params: List[torch.Tensor], next_gradients: List[torch.Tensor]):
    # g[i] corresponds to param[i]    
    for i, param in enumerate(params):
        g = optimizer.current_gradients[i]
        g_next = next_gradients[i]
        if i == 0:
            print(param)
            #print(g)
            #print(g_next)
        
        correction_term = None # to be implemented
        corrected_gradient = None # to be implemented
        
        
    # update all attributes in optimizer
    optimizer.gradients = g
    # ...
    pass

In [8]:
import itertools
from more_itertools import peekable

def adasecant_dataloader(dataset, batch_size, shuffle=False, drop_last=False):
    data_loader = peekable(iter(data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)))
    return data_loader

data_loader = adasecant_dataloader(dataset_test, 60, True, True)
for batch in data_loader:
    #print('current', batch['y'])
    try:
        peek = data_loader.peek()
        #print('next', peek['y'])
    except StopIteration:
        pass

## Training

In [9]:
def evaluate_model(model, dataset, batch_size=1000):
    
    data_loader = data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False, drop_last=False)
    loss = 0.0
    correct = 0
    
    for batch in data_loader:
        yhat = model.forward(batch['X'].float().to(device))
        y = batch['y'].long().to(device)
        batch_loss = f_loss(yhat, y)
        loss += batch_loss.item() * len(batch['X'])
        correct += (torch.argmax(yhat, dim=1) == y).float().sum().item()
    accuracy = correct / len(dataset)
    
    return loss, accuracy

def get_scheduler(optimizer, base_lr, max_lr, epochs_per_cycle, len_dataset, batch_size):
    if epochs_per_cycle is None:
        epochs_per_cycle = epochs
    iterations_per_cycle = epochs_per_cycle * (len_dataset // batch_size)
    return torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr, max_lr, step_size_up=iterations_per_cycle / 2)

In [10]:
def train_model(model, dataset, validation_set, batch_size=64, epochs=1, 
                f_loss=F.cross_entropy, epochs_per_cycle=None):
    
    optimizer = AdaSecant(model.parameters())
    lr_history = []
    validation_accuracy = []
    epoch_losses = []
    validation_losses = []
    
    # evaluate initial state of model
    initial_training_loss, _ = evaluate_model(model, dataset)
    epoch_losses.append(initial_training_loss)
    validation_loss, accuracy = evaluate_model(model, validation_set)
    validation_losses.append(validation_loss)
    validation_accuracy.append(accuracy)

    for epoch in range(epochs):
        # training and epoch loss logging
        # drop last to avoid stochastic outliers in gradient update
        data_loader = adasecant_dataloader(dataset=dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        epoch_loss = 0.0
        
        for batch in data_loader:
            
            # prepare adasecant with current gradient
            model.zero_grad()
            yhat = model.forward(batch['X'].float().to(device))
            batch_loss = f_loss(yhat, batch['y'].long().to(device))
            epoch_loss += batch_loss.item() * len(batch['X'])
            batch_loss.backward()
            for param in model.parameters():
                print(param.grad[0])
            optimizer.pre_step()
            
            # run adasecant with next gradient (addiotionally to current gradient)
            model.zero_grad()
            try:
                next_batch = data_loader.peek()
                yhat = model.forward(next_batch['X'].float().to(device))
                batch_loss = f_loss(yhat, next_batch['y'].long().to(device))
                batch_loss.backward()
                for param in model.parameters():
                    print(param.grad[0])
                optimizer.step()
            except:
                # whatever happens if there is no next gradient
                pass
            return
            
        print(f'Epoch {epoch+1}/{epochs} - Loss: {epoch_loss / len(dataset)}')
        epoch_losses.append(epoch_loss)
        
        # calculate validation loss and accuracy
        validation_loss, accuracy = evaluate_model(model, validation_set)
        validation_losses.append(validation_loss)
        validation_accuracy.append(accuracy)
        
    return (np.array(epoch_losses) / len(dataset), 
            np.array(validation_losses) / len(validation_set), 
            validation_accuracy, 
            lr_history)

In [11]:
base_lr = 0.01
max_lr = 0.1
batch_size = 128
epochs = 1
f_opt = AdaSecant
f_loss = F.cross_entropy
cycle = epochs

training_loss, validation_loss, validation_accuracy, lr_history = train_model(model.to(device),
                                                                              dataset_train,
                                                                              dataset_test,
                                                                              batch_size,
                                                                              epochs,
                                                                              f_loss,
                                                                              cycle)

tensor([-0.0131, -0.0083, -0.0117, -0.0160, -0.0050, -0.0126, -0.0009, -0.0135,
        -0.0117, -0.0107], device='cuda:0')
tensor(-0.0231, device='cuda:0')
tensor([ 0.0031,  0.0030,  0.0049, -0.0035,  0.0040,  0.0013,  0.0031, -0.0037,
         0.0078, -0.0028], device='cuda:0')
tensor(-0.0017, device='cuda:0')
tensor([-0.0113, -0.0172, -0.0152, -0.0065, -0.0074, -0.0097, -0.0160, -0.0116,
        -0.0161, -0.0121], device='cuda:0')
tensor(-0.0266, device='cuda:0')
tensor([ 0.0062, -0.0181,  0.0076, -0.0024, -0.0161, -0.0079, -0.0282,  0.0158,
        -0.0148,  0.0109], device='cuda:0')
tensor(-0.0328, device='cuda:0')
current
[tensor([[-0.0113, -0.0172, -0.0152, -0.0065, -0.0074, -0.0097, -0.0160, -0.0116,
         -0.0161, -0.0121],
        [ 0.0007, -0.0051, -0.0033, -0.0104, -0.0051, -0.0069,  0.0081, -0.0011,
          0.0011, -0.0026],
        [ 0.0008, -0.0007, -0.0079, -0.0045, -0.0116, -0.0044, -0.0026,  0.0045,
         -0.0036, -0.0008],
        [-0.0038,  0.0014, -0.0045, 

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
plt.plot(training_loss, label='training loss')
plt.plot(validation_loss, label='validation loss')
plt.xlabel('Epochs')
plt.ylabel('Mean Cross Entropy Loss')
#plt.ylim(0.0, 10)
plt.legend()

In [ ]:
plt.plot(lr_history)

In [ ]:
plt.plot(validation_accuracy)
print(max(validation_accuracy))

In [ ]:
import csv
from google.colab import files

with open('train_loss', 'w') as f:
    write = csv.writer(f)
    write.writerow(training_loss)

with open('val_loss', 'w') as f: 
    write = csv.writer(f)  
    write.writerow(validation_loss)

with open('val_accuracy', 'w') as f: 
    write = csv.writer(f)  
    write.writerow(validation_accuracy)
    
with open('lr_history', 'w') as f: 
    write = csv.writer(f)  
    write.writerow(lr_history)

files.download('train_loss')
files.download('val_loss')
files.download('val_accuracy')
files.download('lr_history')